In [7]:
import numpy as np
import pandas as pd 
import os
import re
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [9]:
podcasts = pd.read_csv('../data/podcasts.csv')
podcasts.head()
podcasts.info()

,uuid,title,image,description,language,categories,website,author,itunes_id
0,8d62d3880db2425b890b986e58aca393,"Ecommerce Conversations, by Practical Ecommerce",http://is4.mzstatic.com/image/thumb/Music6/v4/...,Listen in as the Practical Ecommerce editorial...,English,Technology,http://www.practicalecommerce.com,Practical Ecommerce,874457373
1,cbbefd691915468c90f87ab2f00473f9,Eat Sleep Code Podcast,http://is4.mzstatic.com/image/thumb/Music71/v4...,On the show we’ll be talking to passionate peo...,English,Tech News | Technology,http://developer.telerik.com/,Telerik,1015556393
2,73626ad1edb74dbb8112cd159bda86cf,SoundtrackAlley,http://is5.mzstatic.com/image/thumb/Music71/v4...,A podcast about soundtracks and movies from my...,English,Podcasting | Technology,https://soundtrackalley.podbean.com,Randy Andrews,1158188937
3,0f50631ebad24cedb2fee80950f37a1a,The Tech M&A Podcast,http://is1.mzstatic.com/image/thumb/Music71/v4...,The Tech M&A Podcast pulls from the best of th...,English,Business News | Technology | Tech News | Business,http://www.corumgroup.com,Timothy Goddard,538160025
4,69580e7b419045839ca07af06cf0d653,"The Tech Informist - For fans of Apple, Google...",http://is4.mzstatic.com/image/thumb/Music62/v4...,The tech news show with two guys shooting the ...,English,Gadgets | Tech News | Technology,http://techinformist.com,The Tech Informist,916080498


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 121175 entries, 0 to 121174
Data columns (total 9 columns):
 #   Column       Non-Null Count   Dtype 
---  ------       --------------   ----- 
 0   uuid         121175 non-null  object
 1   title        121173 non-null  object
 2   image        121175 non-null  object
 3   description  119832 non-null  object
 4   language     121175 non-null  object
 5   categories   121175 non-null  object
 6   website      120005 non-null  object
 7   author       118678 non-null  object
 8   itunes_id    121175 non-null  int64 
dtypes: int64(1), object(8)
memory usage: 8.3+ MB


In [10]:
podcasts = podcasts[podcasts.language == 'English']

In [12]:
podcasts[podcasts.title == 'The Dirtbag Diaries']

,uuid,title,image,description,language,categories,website,author,itunes_id
17832,5555bb00695a4207a7629aa0dcf3f7dd,The Dirtbag Diaries,http://is3.mzstatic.com/image/thumb/Music71/v4...,This is adventure. Climbing. Skiing. Hiking. B...,English,Outdoor | Places & Travel | Society & Culture ...,http://www.dirtbagdiaries.com,Duct Tape Then Beer,218290471


In [ ]:
podcasts = podcasts.dropna(subset=['description'])
podcasts = podcasts.drop_duplicates('itunes_id')
sum(podcasts.description.isnull())

In [ ]:
podcasts['description_length'] = [len(x.description.split()) for _, x in podcasts.iterrows()]
podcasts['description_length'].describe()

In [ ]:
podcasts = podcasts[podcasts.description_length >= 20]

In [6]:
podcasts.shape

NameError: name 'podcasts' is not defined

In [50]:
favorite_podcasts = ['99% Invisible']
favorites = podcasts[podcasts.title.isin(favorite_podcasts)]
favorites

,uuid,title,image,description,language,categories,website,author,itunes_id,description_length
53021,938de7f853b24bf59c7d199b8fba3950,99% Invisible,http://is5.mzstatic.com/image/thumb/Music19/v4...,"\n Design is everywhere in our lives, per...",English,Design | Arts,http://99percentinvisible.org,Roman Mars,394775318,55


In [3]:
podcasts = podcasts[~podcasts.isin(favorites)]
data = pd.concat([podcasts, favorites], sort = True).reset_index(drop = True)

NameError: name 'podcasts' is not defined

In [52]:
data['description'] = data['description'].astype(str)

In [53]:
%%time
tf = TfidfVectorizer(analyzer = 'word', ngram_range = (1, 3), min_df = 0, stop_words = "english")
tf_idf = tf.fit_transform(data['description'])
tf_idf

CPU times: user 38.4 s, sys: 2.37 s, total: 40.8 s
Wall time: 47.4 s


<59323x2944622 sparse matrix of type '<class 'numpy.float64'>'
	with 5380726 stored elements in Compressed Sparse Row format>

In [ ]:
# cosine_similarity dense = False hyperparameter 

In [56]:
%%time
similarity = cosine_similarity(tf_idf, tf_idf, dense_output=False)
similarity

CPU times: user 1min 40s, sys: 22.3 s, total: 2min 2s
Wall time: 2min 27s


<59323x59323 sparse matrix of type '<class 'numpy.float64'>'
	with 1594309221 stored elements in Compressed Sparse Row format>

In [57]:
similarity.shape

(59323, 59323)

In [2]:
x = data[data.title == '99% Invisible'].index[0]
podcast = similarity[x].todense()
similar_idx = podcast.argsort()[-4:-1]
for i in similar_idx:
    print(similarity[x][i], '-', data.title[i], '-', data.description[i], '\n')
print('Original - ' + data.title[x] + '-' + data.description[x])

NameError: name 'data' is not defined

In [64]:
podcast.argsort()[-4:-1]

matrix([], shape=(0, 59323), dtype=int64)

In [19]:
x = data[data.title == 'The Dirtbag Diaries'].index[0]
similar_idx = similarity[x].argsort(axis = 0)[-4:-1]
for i in similar_idx:
    print(similarity[x][i], '-', data.title[i], '-', data.description[i], '\n')
print('Original - ' + data.description[x])

0.0444626602686513 - Adventure Begins Podcast - We are four travel planners coming together to share what we love most - travel! We have a passion for exploring new places, and we love sharing our adventures with friends. So pack your bags, and come along with us! 

0.04691651177621761 - FITz & Healthy Podcast - The FITz and Healthy podcast is a talk show that shares weekly information to help you live a FITz and healthy life.  Join hosts Dr. Lauren Fitz and Cinthanie as they discuss topics ranging from fitness, health, and everything in between to help promote a better healthy lifestyle by design.  The FITz and Healthy podcast offer a series of shows on the official YouTube channel and other social sites. 

0.07087294652490107 - KSL Outdoors Show - Fishing, hunting, camping, hiking, biking, offroading, four-wheeling, ATVs, dutch ovens - if you can do it outside, we're all over it. 

Original - This is adventure. Climbing. Skiing. Hiking. Biking. Travel. Whatever your passion, we are a